In [1]:
from utils import *
from datetime import datetime
import pandas as pd
import numpy as np

### 1. Setup

Leitura da base de dados de Ações e outra de Bdrs

In [2]:
df_bdrs = pd.read_excel('Datasets/Tickers_Bovespa.xlsx', 'BDRs', 0)
df_acoes = pd.read_excel('Datasets/Tickers_Bovespa.xlsx', 'Ações', 0)

C:\Users\panto\AppData\Local\Temp/ipykernel_1564/4102752140.py:1: FutureWarning: Starting with pandas version 2.0 all arguments of read_excel except for the arguments 'io' and 'sheet_name' will be keyword-only
  df_bdrs = pd.read_excel('Datasets/Tickers_Bovespa.xlsx', 'BDRs', 0)
C:\Users\panto\AppData\Local\Temp/ipykernel_1564/4102752140.py:2: FutureWarning: Starting with pandas version 2.0 all arguments of read_excel except for the arguments 'io' and 'sheet_name' will be keyword-only
  df_acoes = pd.read_excel('Datasets/Tickers_Bovespa.xlsx', 'Ações', 0)


In [ ]:
df_bdrs.head()

In [ ]:
df_acoes.head()

### 2. Pré-Processamento 1.1.

A base de dados teve de ser rearranjada para formar uma lista de ações (Um Ativo por registro).

In [ ]:
df_acoes = df_acoes.melt(id_vars=['Empresas'], value_vars=['Ativos_1', 'Ativos_2', 'Ativos_3', 'Ativos_4', 'Ativos_5', 'Ativos_6'], value_name='Ativos').drop('variable', axis=1).dropna().reset_index(drop=True)

In [ ]:
df_acoes.head()

### 3. Pré-Processamento 1.2.

A base de dados de Ações e Bdrs devem ser concatenadas, para isso:

1. Uma indicação de origem deve ser criado, a nova coluna tipo pode ter valor 'bdrs' ou 'acoes';
2. 'SETOR' e 'PAÍS DE ORIGEM', devem ser descartados;
3. A colunas devem coincidir em tipo e nome;
4. Então podem ser concatenadas;
5. O indíce deve ser resetado.

In [ ]:
df_bdrs['Tipo'] = 'bdrs'
df_acoes['Tipo'] = 'acoes'

In [ ]:
df_bdrs.drop(['SETOR', 'PAÍS DE ORIGEM'], inplace=True, axis=1)

In [ ]:
df_bdrs.rename(columns={'EMPRESA': 'Empresas', 'CÓDIGO': 'Ativos'}, inplace=True)

In [ ]:
df_bdrs.head()

In [ ]:
df = pd.concat([df_acoes, df_bdrs]).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df.tail()

### 4. Primeira Iteração de coleta de dados do website Status Invest

Coleta dados fundamentalistas da lista de ativos de df e armazena em prices_data. Os casos com erros (stock_errors) são mapeados em df, na coluna 'error'.

In [ ]:
scraper = StatusInvestScraper()

In [ ]:
from_date = datetime(2011, 1, 1)
to_date = datetime(2022, 6, 1)
interval = '1mo'

In [ ]:
stock_errors = []
fundamental_data = None

for idx, data in df.iterrows():
    ticker = data['Ativos'].strip()
    try:
        stock_data = scraper.get_data(data['Tipo'], ticker, from_date, to_date, interval)
        stock_data['Ticker'] = data['Ativos']
        stock_data['Tipo'] = data['Tipo']
        if fundamental_data is None:
            fundamental_data = stock_data
        else:
            fundamental_data = pd.concat([fundamental_data, stock_data])
        print("log {}/{}: Success".format(idx,df.shape[0]))
    except:
        stock_errors.append(idx)
        print("log {}/{}: Error -> {}".format(idx,df.shape[0], scraper.url))

df['error'] = 0
df['error'].iloc[stock_errors] = 1

In [ ]:
fundamental_data.head()

In [ ]:
df.head()

### 5. Segunda Iteração de coleta de dados do website Status Invest

Na primeira iteração, notou-se que alguns ativos estão com último caractere 'F' na lista, mas no Status Invest havia ativos semelhantes sem este sufixo. Dessa forma, tentei sem o último caractere.

In [ ]:
stock_errors = []
for idx, data in df.iterrows():
    if not data['error']:
        continue
        
    ticker = data['Ativos'].strip()
    ticker = ticker[:-1]
    try:
        stock_data = scraper.get_data(data['Tipo'], ticker, from_date, to_date, interval)
        stock_data['Ticker'] = ticker
        stock_data['Tipo'] = data['Tipo']
        fundamental_data = pd.concat([fundamental_data, stock_data])
        print("log {}/{}: Success".format(idx,df.shape[0]))
    except:
        stock_errors.append(idx)
        print("log {}/{}: Error -> {}".format(idx,df.shape[0], scraper.url))

In [ ]:
scraper.close()

In [ ]:
fundamental_data.to_csv('Datasets/dados_fudamentalistas.csv', index=False)

In [ ]:
df.to_csv('Datasets/dados_intermediarios.csv', index=False)